In [16]:
import numpy as np
import os
import csv
from typing import List, Dict, Tuple
import h5py
from collections import defaultdict
import matplotlib.pyplot as plt



In [9]:
pbs_example = '/mnt/hd0/Pain_ML_data/videos/pbs/pbs-trimmed_2024-07-15_22-04-39_1-2blue'
formalin_example = '/mnt/hd0/Pain_ML_data/videos/formalin/formalin-trimmed_2024-07-15_22-04-39_1-1blue'

ex = os.path.join(pbs_example,'features.h5')

In [10]:
# def traverse_hdf5(name, node, group_data):
#     group_name = name.split('/')[0]  # Get the top-level group name
#     if isinstance(node, h5py.Dataset):
#         if group_name not in group_data:
#             group_data[group_name] = {}
#         dataset_name = name.split('/')[-1]  # Get the dataset name
#         group_data[group_name][dataset_name] = {
#             'shape': node.shape,
#             'dtype': str(node.dtype),
#             # 'sample': node[(0,) * len(node.shape)][:5].tolist()  # Convert to list for easier printing
#         }

# def explore_h5_file(file_path):
#     group_data = defaultdict(dict)
#     with h5py.File(file_path, 'r') as f:
#         f.visititems(lambda name, node: traverse_hdf5(name, node, group_data))
    
#     print(f"Exploring file: {file_path}\n")
#     for group_name, datasets in group_data.items():
#         print(f"Group: {group_name}")
#         print(f"Number of datasets: {len(datasets)}")
#         total_data_points = sum(info['shape'][0] for info in datasets.values())
#         print(f"Total data points: {total_data_points}")
#         print("Datasets:")
#         for dataset_name, info in datasets.items():
#             print(f"  - {dataset_name}")
#             print(f"    Shape: {info['shape']}")
#             print(f"    Type: {info['dtype']}")
#             # print(f"    First few values: {info['sample']}")
#         print()
#     return group_data

# explore_h5_file(ex)

Exploring file: /mnt/hd0/Pain_ML_data/videos/pbs/pbs-trimmed_2024-07-15_22-04-39_1-2blue/features.h5

Group: pbs-trimmed_2024-07-15_22-04-39_1-2blue
Number of datasets: 27


IndexError: tuple index out of range

In [17]:
class HDF5Manager:
    def __init__(self, file_path):
        self.file_path = file_path
        self.file = None
        self.group_data = defaultdict(dict)
        self._traverse_file()

    def _traverse_file(self):
        with h5py.File(self.file_path, 'r') as f:
            f.visititems(self._collect_datasets)

    def _collect_datasets(self, name, node):
        if isinstance(node, h5py.Dataset):
            group_name = name.split('/')[0]
            dataset_name = name.split('/')[-1]
            self.group_data[group_name][dataset_name] = name  # Store full path

    def _ensure_file_open(self):
        if self.file is None or not self.file.id.valid:
            self.file = h5py.File(self.file_path, 'r')

    def close(self):
        if self.file is not None:
            self.file.close()
            self.file = None

    def get_dataset(self, group_name, dataset_name) -> np.array:
        # Extracts given dataset from a specific group
        self._ensure_file_open()
        full_path = self.group_data[group_name][dataset_name]
        return self.file[full_path]

    def get_dataset_for_all_groups(self, dataset_name) -> Dict[str,np.array]:
        dataset_dict = {}
        for group_name in self.group_data.keys():
            dataset = self.get_data(group_name, dataset_name)
            dataset_dict[group_name] = dataset
        return dataset_dict

    def get_data(self, group_name, dataset_name, slice=None):
        dataset = self.get_dataset(group_name, dataset_name)
        if slice is None:
            return dataset[:]
        return dataset[slice]

    def get_metadata(self, group_name, dataset_name):
        dataset = self.get_dataset(group_name, dataset_name)
        return {
            'shape': dataset.shape,
            'dtype': str(dataset.dtype),
            'chunks': dataset.chunks,
            'compression': dataset.compression
        }

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

    def __del__(self):
        self.close()


In [25]:
file_path = ex
with HDF5Manager(file_path) as hdf:
    # List all groups
    print("Groups:", list(hdf.group_data.keys()))
    # Choose a group
    group_name = list(hdf.group_data.keys())[0]
    print(f"\nDatasets in {group_name}:", list(hdf.group_data[group_name].keys()))
    # Get data
    data = hdf.get_data(group_name, 'both_front_paws_lifted')

Groups: ['pbs-trimmed_2024-07-15_22-04-39_1-2blue']

Datasets in pbs-trimmed_2024-07-15_22-04-39_1-2blue: ['ankle_distance', 'background_luminance', 'both_front_paws_lifted', 'cheek_distance', 'chest_head_angle', 'distance_delta', 'fps', 'frame_count', 'front_left_luminance', 'front_paws_distance', 'front_right_luminance', 'hind_left_luminance', 'hind_paws_distance', 'hind_right_luminance', 'hip_chest_angle', 'hip_sternumtail_distance', 'hip_tailbase_distance', 'hip_tailbase_hlpaw_angle', 'hip_tailbase_hrpaw_angle', 'hip_width', 'neck_snout_distance', 'paw_guarding', 'shoulder_width', 'sternumhead_neck_distance', 'sternumtail_sternumhead_distance', 'tail_hip_angle', 'tailbase_tailtip_distance']


In [26]:
data

array([False, False, False, ..., False, False, False])

In [24]:
np.unique(data, return_counts=True)

(array([False,  True]), array([13206,   117]))

In [27]:
pain_datasets = ['paw_guarding','both_front_paw_lifted'] #potentially add ankle distance 
# add luminance_logratio after rerunning analysis-public on all recordings
